# 06-14

- KNN Clustering 
- Using Categories from KNN to get subset from information
- Continous Features(Transition and Highlevel features)
- Categorical Features
- MinMax Scaling to all continous and categorical
- Remove Constant Features continous and categorical
- Gower

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt 
import math
from ipynb.fs.full.Utils import createKDeplot, loadDataFrames, dropFeaturesWithNoVariance,loadCoverGroups
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy import stats
import dask_ml.cluster
import pandas as pd
import gower
from ipynb.fs.full.Pipeline import removeConstantFeatures, applyBoxCoxAllFeatures, centerAndScale, applyEuclideanDistance, getIndexAndVector, applyBoxCoxFeatures,applyJaccard, getEuclideanJaccardDf , applyCosineDistance, applyGower

In [2]:
cover = loadCoverGroups()
listCovers = cover[6]
originalSongId = listCovers[0]
coverSongId = listCovers[1]

In [3]:
transitionMatrices = pd.read_csv('./data/unified/07-transitionsClustered.csv')
transitionMatrices = transitionMatrices[['id','category']]

In [4]:
df = pd.read_csv('./data/unified/05-allDataContinousCategoricalTransition.csv')
ids = df.id
df = df.iloc[:,2:]

In [5]:
df

,id,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
0,23,0.494,0.756,0.0432,0.001400,0.1640,0.589,133.337,0.000000,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,105,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344,0.185185,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,118,0.614,0.755,0.0276,0.016200,0.0831,0.645,92.622,0.000000,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120,0.319,0.807,0.0388,0.001040,0.1310,0.601,118.097,0.029412,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,292,0.496,0.527,0.0339,0.195000,0.1380,0.283,155.932,0.013889,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5972,6002496,0.509,0.537,0.0384,0.347000,0.0805,0.311,170.030,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5973,6002497,0.725,0.583,0.0427,0.300000,0.0665,0.762,79.984,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5974,6002498,0.399,0.787,0.0499,0.019700,0.0685,0.572,117.089,0.000000,0.015385,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5975,6002499,0.376,0.435,0.0264,0.513000,0.1380,0.304,80.002,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [6]:
df = pd.merge(df,transitionMatrices, on='id')

In [7]:
categoryOriginalSong = df[df.id == originalSongId].category.values[0]
df = df[df.category == categoryOriginalSong]
df = df.drop(columns=['category'])

In [8]:
## Removing unamed and other unnecessary columns
df.reset_index(drop=True, inplace=True)
df = df.sort_values('id')
df.id = df.id.astype('int64')
# Saving idChords
ids = df.id
df = df.set_index('id')
df.reset_index(drop=True, inplace=True)
df

,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,2,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
0,0.494,0.756,0.0432,0.001400,0.1640,0.589,133.337,0.000000,0.000000,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.319,0.807,0.0388,0.001040,0.1310,0.601,118.097,0.029412,0.000000,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.511,0.517,0.0328,0.130000,0.0740,0.433,75.670,0.000000,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.330,0.905,0.0497,0.000233,0.0712,0.259,163.142,0.104478,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.311,0.815,0.0443,0.253000,0.6080,0.700,147.173,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2420,0.713,0.798,0.2190,0.366000,0.2460,0.721,176.073,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2421,0.725,0.583,0.0427,0.300000,0.0665,0.762,79.984,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2422,0.399,0.787,0.0499,0.019700,0.0685,0.572,117.089,0.000000,0.015385,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2423,0.376,0.435,0.0264,0.513000,0.1380,0.304,80.002,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [9]:
df = removeConstantFeatures(df)
#df= applyBoxCoxFeatures(df, ['speechiness', 'acousticness', 'liveness', 'valence'])
#df= applyBoxCoxAllFeatures(df)
#df = centerAndScale(df)

In [10]:
scaler = MinMaxScaler()
scaler.fit(df)
df = pd.DataFrame(scaler.transform(df)).copy()

In [11]:
ranksDFGower = applyGower(df,ids, originalSongId,len(df))
ranksDFGower.head(10)

,id,values
0,6001931,0.000000
1,6001754,0.004440
2,6000473,0.006452
3,6000143,0.006585
4,6000054,0.006611
5,6000152,0.006729
6,6001662,0.007041
7,6001689,0.007141
8,6001902,0.007160
9,6002481,0.007499


In [12]:
ranksDFGower.loc[ranksDFGower.id.isin(listCovers)]

,id,values
0,6001931,0.000000
354,6001748,0.011149


In [13]:
ranksDFGower[ranksDFGower['id']==coverSongId]

,id,values
354,6001748,0.011149
